In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 butsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import(
    AutoModelForCasualLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer


QLoRA uses a rank of 64 with a scaling parameter of 16. Loading the Llama-2 model directly in 4-bit precision sing the NF4 type and train it for one epoch

In [ ]:
# Model that we want to train with the Hugging Face Hub
model_name = "NousResearch/Llama-2-7b-chat-hf"

# Instruction dataset
dataset_name = "mlabonne/guanaco-llama2-1k"

# New Fine-tuned model name
new_model = "Llama-2-7b-chat-finetune"

# LoRA attention dimensions
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability
lora_dropout = 0.1

###---------------------###
# BitsAndBytes parameters #
###---------------------###

# Activate 4-bit precision base model 
use_4bit = True

# Compute dtype for 4-bit base model
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quatnization for 4-bit base model (double quantization)
use_nested_quant = False


###------###
# Training #
###------###

output_dir = "./results"

num_training_epochs = 1

# Enable fp16/bf16 training 
fp16 = False
bf16 = False

# Batch size for training and evaluation
per_device_train_batch_size = 4
per_device_eval_batch_size = 4

# Num of update steps to accumulate the gradient for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True


# Max gradient normal (Gradient clipping)
max_grad_norm = 0.3

# Initial learining rate (AdamW)
learning_rate = 2e-4

# Weight decay
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# LR scheduler
lr_scheduler_type = "cosine"

# Number of raining steps (overrides num_training_epochs)
max_step = -1

# Ratio of steps fo a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training conisderably
group_by_lentgh = True

# Save checkpoint every X updates steps
save_steps = 0

# Log ecery X updates steps
logging_steps = 25

###------------###
# SFT Parameters #
###------------###

# Max sequnce length to use
max_seq_length = None

# Pack multiple short examples into the same sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}


In [ ]:
dataset = load_dataset(dataset_name, split="train")



# tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

# reating a BitsAndBytesConfig object with specific settings for quantization, precision and optimization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,  # Enable 4-bit quantization
    bnb_4bit_quant_type=bnb_4bit_quant_type,    # use certain quantization method
    bnb_4bit_compute_dtype=compute_dtype,       # compute precision
    bnb_4bit_use_duble_quant=use_nested_quant   # Double quantization true/false
)       # the configuration is then passed to the model when loading it


# Check if GPU is compatible with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load the base model
model = AutoModelForCasualLM.from_pretrained(
    model_name, quantization_config = False, device_map = device_map
)
model.config.use_cache = False          # controls whether the model caches past key-value states for attention layers
model.config.pretrained_tp = 1          # Does the model run on single GPU or multi-GPU training

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    # some models define custom tokenization logic in their model repo
    # trust_remote_code allows to execte a custom tokenizer code form hugging face
    model_name, trust_remote_code = True
)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"


# Load LoRA configuration
# settings for LoRA fine-tuning
peft_config = LoraConfig(
    lora_alpha=lora_alpha,          # scaling factor that adjusts LoRA weights. Higher values increase learning stability
    lora_dropout=lora_dropout,      # Dropout probability
    r=lora_r,                       # LoRA rank - contorls the number of traninable parameters, lower value reduces memory usage
    bias="none",                    # if LoRA modifies the bias term
    task_type="CAUSAL_LM"           # Defines the task type
)

# Traning parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_training_epochs=num_training_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_step=max_step,
    warmup_ratio=warmup_ratio,
    group_by_lentgh=group_by_lentgh,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing
)

trainer.train()

# Save the trained model
trainer.model.save_retrained(new_model)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir results/runs

In [ ]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generatio pipeline
prompt = "What is an LLM?"          #<- query for the model
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt}[/INST]")
print(result[0]["generated_text"])